[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/pixeltable/pixeltable/blob/master/docs/release/tutorials/rag-operations.ipynb)&nbsp;&nbsp;
[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pixeltable/pixeltable/blob/master/docs/release/tutorials/rag-operations.ipynb)

# RAG Operations in Pixeltable

In this tutorial, we'll explore Pixeltable's flexible handling of RAG operations on unstructured text. In a traditional AI workflow, such operations might be implemented as a Python script that runs on a periodic schedule or in response to certain events. In Pixeltable, as with everything else, they are implemented as persistent table operations that update incrementally as new data becomes available. In our tutorial workflow, we'll chunk Wikipedia articles in various ways with a document splitter, then apply several kinds of embeddings to the chunks.

To run this tutorial, you'll need to have Pixeltable installed, along with the Huggingface `sentence_transformers` package.

## Set Up the Table Structure

We start by creating a Pixeltable workspace `rag_demo` (if it doesn't already exist) and setting up the table structure for our new workflow.

In [1]:
import pixeltable as pxt

# Create the Pixeltable workspace
pxt.create_dir('rag_demo', ignore_errors=True)

# Clean the database to ensure we're using fresh table instances
# (in case this demo has been run before)
pxt.drop_table('rag_demo.short_char_chunks', ignore_errors=True)
pxt.drop_table('rag_demo.short_chunks', ignore_errors=True)
pxt.drop_table('rag_demo.chunks', ignore_errors=True)
pxt.drop_table('rag_demo.sentences', ignore_errors=True)
pxt.drop_table('rag_demo.docs', ignore_errors=True)

Connected to Pixeltable database at: postgresql://postgres:@/pixeltable?host=/Users/asiegel/.pixeltable/pgdata
Created directory `rag_demo`.


## Creating Tables and Views

Now we'll create the tables that represent our workflow, starting with a table to hold references to source documents. The table contains a single column `source_doc` whose elements have type `pxt.DocumentType`, representing a general document instance. In this tutorial, we'll be working with HTML documents, but Pixeltable supports a range of other document types, such as Markdown and PDF.

In [2]:
docs = pxt.create_table('rag_demo.docs', {'source_doc': pxt.DocumentType()})

Created table `docs`.


If we take a peek at the `docs` table, we see its very simple structure.

In [3]:
docs

Column Name,Type,Computed With
source_doc,document,


Next we create a view to represent chunks of our HTML documents. A Pixeltable view is a virtual table, which is dynamically derived from a source table by applying a transformation and/or selecting a subset of data. In this case, our view represents a one-to-many transformation from source documents into individual sentences. This is achieved using Pixeltable's built-in `DocumentSplitter` class.

Note that the `docs` table is currently empty, so creating this view doesn't actually *do* anything yet: it simply defines an operation that we want Pixeltable to execute when it sees new data.

In [5]:
from pixeltable.iterators.document import DocumentSplitter

sentences = pxt.create_view(
    'rag_demo.sentences',  # Name of the view
    docs,  # Table from which the view is derived
    iterator=DocumentSplitter.create(
        document=docs.source_doc,
        separators='sentence',  # Chunk docs into sentences
        metadata='title,heading,sourceline'
    )
)

Created view `sentences` with 0 rows, 0 exceptions.


Let's take a peek at the new `sentences` view. Run the following command:

In [6]:
sentences

Column Name,Type,Computed With
pos,int,
text,string,
title,string,
heading,json,
sourceline,int,
source_doc,document,


We see that `sentences` inherits the `source_doc` column from `docs`, together with some new fields:
- `pos`: The position in the source document where the sentence appears.
-  `text`: The text of the sentence.
- `title`, `heading`, and `sourceline`: The metadata we requested when we set up the view.

## Data Ingestion

Ok, now it's time to insert some data into our workflow. A document in Pixeltable is just a URL; the following command inserts a single row into the `docs` table with the `source_doc` field set to the specified URL:

In [7]:
docs.insert(source_doc='https://en.wikipedia.org/wiki/Marc_Chagall')

Inserting rows into `docs`: 1 rows [00:00, 389.73 rows/s]
Inserting rows into `sentences`: 6 rows [00:00, 5406.19 rows/s]
Inserted 7 rows with 0 errors.


UpdateStatus(num_rows=7, num_computed_values=0, num_excs=0, updated_cols=[], cols_with_excs=[])

We can see that two things happened. First, a single row was inserted into `docs`, containing the URL representing our source document. Then, the view `sentences` was incrementally updated by applying the `DocumentSplitter` according to the definition of the view. This illustrates an important principle in Pixeltable: by default, anytime Pixeltable sees new data, the update is incrementally propagated to any downstream views or computed columns.

We can see the effect of the insertion with the `select` command. There's a single row in `docs`:

In [8]:
docs.select(docs.source_doc.fileurl).show()

source_doc_fileurl
https://en.wikipedia.org/wiki/Marc_Chagall


And here are the first 20 rows in `sentences`. The content of the article is broken into individual sentences, as expected.

In [9]:
sentences.select(sentences.text, sentences.heading).show(20)

text,heading
Main menu,{}
Search Search,{}
Personal tools,{}
Toggle the table of contents,{}
81 languages,{}
Toggle limited content width,{}


## Experimenting with Chunking

Of course, chunking into sentences isn't the only way to split a document. Perhaps we want to experiment with different chunking methodologies, in order to see which one performs best in a particular application. Pixeltable makes it easy to do this, by creating several views of the same source table. Here are a few examples. Notice that as each new view is created, it is initially populated from the data already in `docs`.

In [11]:
chunks = pxt.create_view(
    'rag_demo.chunks', docs,
    iterator=DocumentSplitter.create(
        document=docs.source_doc,
        separators='paragraph,token_limit',
        limit=2048,
        overlap=0,
        metadata='title,heading,sourceline'
    )
)

Inserting rows into `chunks`: 6 rows [00:00, 5545.58 rows/s]
Created view `chunks` with 6 rows, 0 exceptions.


In [12]:
short_chunks = pxt.create_view(
    'rag_demo.short_chunks', docs,
    iterator=DocumentSplitter.create(
        document=docs.source_doc,
        separators='paragraph,token_limit',
        limit=72,
        overlap=0,
        metadata='title,heading,sourceline'
    )
)

Inserting rows into `short_chunks`: 6 rows [00:00, 5550.47 rows/s]
Created view `short_chunks` with 6 rows, 0 exceptions.


In [13]:
short_char_chunks = pxt.create_view(
    'rag_demo.short_char_chunks', docs,
    iterator=DocumentSplitter.create(
        document=docs.source_doc,
        separators='paragraph,char_limit',
        limit=72,
        overlap=0,
        metadata='title,heading,sourceline'
    )
)

Inserting rows into `short_char_chunks`: 6 rows [00:00, 5268.12 rows/s]
Created view `short_char_chunks` with 6 rows, 0 exceptions.


In [15]:
chunks.select(chunks.text, chunks.heading).show(20)

text,heading
Main menu,{}
Search Search,{}
Personal tools,{}
Toggle the table of contents,{}
81 languages,{}
Toggle limited content width,{}


In [16]:
short_chunks.select(short_chunks.text, short_chunks.heading).show(20)

text,heading
Main menu,{}
Search Search,{}
Personal tools,{}
Toggle the table of contents,{}
81 languages,{}
Toggle limited content width,{}


In [17]:
short_char_chunks.select(short_char_chunks.text, short_char_chunks.heading).show(20)

text,heading
Main menu,{}
Search Search,{}
Personal tools,{}
Toggle the table of contents,{}
81 languages,{}
Toggle limited content width,{}


Now let's add a few more documents to our workflow. Notice how all of the downstream views are updated incrementally, processing just the new documents as they are inserted.

In [18]:
urls = [
    'https://en.wikipedia.org/wiki/Pierre-Auguste_Renoir',
    'https://en.wikipedia.org/wiki/Henri_Matisse',
    'https://en.wikipedia.org/wiki/Marcel_Duchamp'
]
docs.insert({'source_doc': url} for url in urls)

Inserting rows into `docs`: 3 rows [00:00, 3456.84 rows/s]
Inserting rows into `sentences`: 18 rows [00:00, 13025.79 rows/s]
Inserting rows into `chunks`: 18 rows [00:00, 13402.71 rows/s]
Inserting rows into `short_chunks`: 18 rows [00:00, 14708.25 rows/s]
Inserting rows into `short_char_chunks`: 18 rows [00:00, 12153.49 rows/s]
Inserted 75 rows with 0 errors.


UpdateStatus(num_rows=75, num_computed_values=0, num_excs=0, updated_cols=[], cols_with_excs=[])

## Further Experiments

This is a good time to mention another important guiding principle of Pixeltable. The preceding examples all used the built-in `DocumentSplitter` class with various configurations. That's probably fine as a first cut or to prototype an application quickly, and it might be sufficient for some applications. But other applications might want to do more sophisticated kinds of chunking, implementing their own specialized logic or leveraging third-party tools. Pixeltable imposes no constraints on the AI or RAG operations a workflow uses: the iterator interface is highly general, and it's easy to implement new operations or adapt existing code or third-party tools into the Pixeltable workflow. Pixeltable manages *data* storage and workflow that is common to all such operations; above the data plane, it's fully general and extensible.

TODO: Example of a custom splitter?

## Adding embeddings

Next, let's look at how embeddings can be added seamlessly to existing Pixeltable workflows. To compute our embeddings, we'll use the Huggingface `sentence_transformer` package, running it over the `chunks` view that broke our documents up into larger paragraphs. Pixeltable has a built-in `sentence_transformer` adapter, and all we have to do is add a new column that leverages it. Pixeltable takes care of the rest, applying the new column to all existing data in the view.

In [21]:
from pixeltable.functions.huggingface import sentence_transformer

minilm_model_id = 'paraphrase-MiniLM-L6-v2'

chunks['minilm_embedding'] =sentence_transformer(chunks.text, model_id=minilm_model_id)

Computing cells: 100%|█████████████████████████████████████████| 24/24 [00:00<00:00, 322.89 cells/s]
Added 24 column values with 0 errors.


The new column is a *computed column*: it is defined as a function on top of existing data and updated incrementally as new data are added to the workflow. Let's have a look at how the new column affected the `chunks` view.

In [22]:
chunks

Column Name,Type,Computed With
pos,int,
text,string,
title,string,
heading,json,
sourceline,int,
minilm_embedding,"array((384,), dtype=FLOAT)","sentence_transformer(text, model_id='paraphrase-MiniLM-L6-v2')"
source_doc,document,


In [23]:
chunks.head()

pos,text,title,heading,sourceline,minilm_embedding,source_doc
0,Main menu,,{},0,"[0.013837013, 0.14026177, 0.28270477, -0.08148003, -0.49137008, 0.23778768, -0.25082362, -0.23600236, 0.47489968, -0.27540472, -0.7250494, -0.04896789, -0.17751567, -0.60430855, -0.10019561, -1.0068173, 1.0703558, -0.45004058, 0.4654496, -0.14810751, 0.13316727, -0.10193365, -0.8039463, 0.039722055, 0.0030924305, -0.054789938, -0.08696906, 0.6339634, -0.18833934, -0.72005594, 0.045729116, -0.01488281, 0.63569754, -0.019973017, -0.08143306, -0.227109, -0.23942484, -0.5756098, -0.2401254, -0.117910385, -0.13598639, 0.6119604, -0.46512628, 0.20938733, 0.050053112, -0.35243317, -0.26929402, -0.1806831, 0.42443514, 0.36750057, -0.30398357, 0.5808253, -0.2442749, 0.062205933, 0.4597018, 0.74200726, -0.28308427, 0.17852956, -0.099436596, 0.32848102, -0.023009568, -0.118511856, -0.37992948, 0.114756465, 0.94367504, 0.4259227, -0.214038, 0.6544014, -0.62131244, 0.105964944, -0.5896095, -0.31305873, 0.25245723, 0.395325, 0.58011234, -0.3402485, -0.098590426, -0.3611181, -0.85350585, 0.3868234, 0.23653977, 0.09506751, -0.35156268, -0.64272165, 0.6753231, -0.21587954, -0.90433544, -0.46418548, -0.24809398, -0.27525756, 0.0915171, -0.27818486, 0.38402498, 0.12188502, 0.4549827, -0.2626203, -0.14732328, -1.1077989, 0.24514978, 1.0587058, ...]",/Users/asiegel/.pixeltable/file_cache/26f958802a424414b408646538f72715_0_3c5b8031c7610e17a42ab6df79e614c2ee1a85dbd022497373d574cf65e15c51
1,Search Search,,{},0,"[-0.4388815, 0.2616052, -0.09626006, 0.27324373, 0.32822007, 0.16367495, 0.45697403, -0.35676295, 0.134427, -0.1315698, 0.8494229, -0.107490435, -0.015755985, 0.112038106, -0.07407861, -0.6957965, 0.14700912, 0.06417219, 0.40219134, -0.23525691, 0.51686716, 0.096137054, -0.31232864, -0.5254353, 0.048178654, -0.31934243, -0.6444056, -0.48666033, 0.52254075, -0.3103736, 0.3446966, -0.21178645, 0.23859325, 0.10162328, 0.8573472, -0.18241832, -0.34733444, -0.20165437, 0.15406382, -0.19690192, 0.10764988, -0.30204758, -0.06345734, 0.8638339, -0.037420724, -0.022490362, -0.6369642, 0.4802386, 0.29758453, 0.42319107, -0.8741574, -0.4897521, -0.45491284, -0.12441757, 0.07507527, 0.23504484, -0.07021256, -0.17961068, 0.2916925, -0.48828724, 0.9852896, 0.26137888, -0.7030126, 0.31352597, -0.005079925, 0.052201174, -0.1211284, 0.006912276, 0.40216282, 0.22954431, 0.5131579, 0.26010892, -0.85585743, 0.7217986, -0.8300178, 0.31207216, -0.07977496, -0.49203598, -0.018743463, -0.31410235, -0.2637596, -0.31347972, 0.0461054, 0.13912694, -0.08210782, -0.28269756, 0.22598626, 0.32890493, 0.6892837, -0.17732784, -0.13792439, -0.44667035, -0.5042677, -0.42169738, -0.88269377, -0.058416136, 0.114198565, 0.17162606, 0.64845335, 1.0464563, ...]",/Users/asiegel/.pixeltable/file_cache/26f958802a424414b408646538f72715_0_3c5b8031c7610e17a42ab6df79e614c2ee1a85dbd022497373d574cf65e15c51
2,Personal tools,,{},0,"[-1.0809782, 0.14871615, -0.17175414, -0.16569878, 0.21554732, -0.03357071, 0.4761501, 0.23835604, 0.19804165, -0.08035711, -0.32521123, -0.009721253, 0.23285279, -0.13696082, -0.06707458, -0.16528586, -0.058667198, 0.7998024, -0.06131527, -0.051864143, 0.5496594, 0.18582836, -0.2659592, -0.031910174, 0.078132525, 0.6470892, -0.28053263, -0.18952954, 0.10174167, -0.8660025, 0.4749312, -0.074961275, 0.050736323, 0.037845034, 0.088446304, 0.2014157, -0.28794533, -0.27062154, 0.07520204, -0.5154939, -0.16101709, 0.18315947, -0.31622407, 0.31491187, 0.72120714, 0.17938687, -0.20015639, -0.07261648, -0.069424845, 0.4641994, -0.37641528, 0.019972838, -0.55050266, -0.0468921, 0.15563634, 0.536762, 0.35447353, -0.052544452, -0.2500377, -0.2003059, 0.64787114, -0.0812662, -0.8306557, 0.6260421, 0.5074377, 0.80805594, -0.14777714, 0.10732351, -0.25287554, -0.19226336, -0.049908377, -0.22145715, -0.30398428, 0.13637014, 0.6990555, -0.3411329, -0.63882506, -0.45870435, -0.52531976, 0.22960809, -0.42343894, 0.9150864, 0.5560932, 0.82766265, 0.3368435, 0.49894124, 0.4110324, 0.24729744, 0.123888955, 

Similarly, we might want to add a CLIP embedding to our workflow; once again, it's just another computed column:

In [24]:
from pixeltable.functions.huggingface import clip_text

clip_model_id = 'openai/clip-vit-base-patch32'

chunks.add_column(clip_embedding=clip_text(text=chunks.text, model_id=clip_model_id))

Computing cells: 100%|██████████████████████████████████████████| 24/24 [00:00<00:00, 24.95 cells/s]
Added 24 column values with 0 errors.


UpdateStatus(num_rows=24, num_computed_values=24, num_excs=0, updated_cols=[], cols_with_excs=[])

In [25]:
chunks

Column Name,Type,Computed With
pos,int,
text,string,
title,string,
heading,json,
sourceline,int,
minilm_embedding,"array((384,), dtype=FLOAT)","sentence_transformer(text, model_id='paraphrase-MiniLM-L6-v2')"
clip_embedding,"array((512,), dtype=FLOAT)","clip_text(text, model_id='openai/clip-vit-base-patch32')"
source_doc,document,


In [26]:
chunks.head()

pos,text,title,heading,sourceline,minilm_embedding,clip_embedding,source_doc
0,Main menu,,{},0,"[0.013837013, 0.14026177, 0.28270477, -0.08148003, -0.49137008, 0.23778768, -0.25082362, -0.23600236, 0.47489968, -0.27540472, -0.7250494, -0.04896789, -0.17751567, -0.60430855, -0.10019561, -1.0068173, 1.0703558, -0.45004058, 0.4654496, -0.14810751, 0.13316727, -0.10193365, -0.8039463, 0.039722055, 0.0030924305, -0.054789938, -0.08696906, 0.6339634, -0.18833934, -0.72005594, 0.045729116, -0.01488281, 0.63569754, -0.019973017, -0.08143306, -0.227109, -0.23942484, -0.5756098, -0.2401254, -0.117910385, -0.13598639, 0.6119604, -0.46512628, 0.20938733, 0.050053112, -0.35243317, -0.26929402, -0.1806831, 0.42443514, 0.36750057, -0.30398357, 0.5808253, -0.2442749, 0.062205933, 0.4597018, 0.74200726, -0.28308427, 0.17852956, -0.099436596, 0.32848102, -0.023009568, -0.118511856, -0.37992948, 0.114756465, 0.94367504, 0.4259227, -0.214038, 0.6544014, -0.62131244, 0.105964944, -0.5896095, -0.31305873, 0.25245723, 0.395325, 0.58011234, -0.3402485, -0.098590426, -0.3611181, -0.85350585, 0.3868234, 0.23653977, 0.09506751, -0.35156268, -0.64272165, 0.6753231, -0.21587954, -0.90433544, -0.46418548, -0.24809398, -0.27525756, 0.0915171, -0.27818486, 0.38402498, 0.12188502, 0.4549827, -0.2626203, -0.14732328, -1.1077989, 0.24514978, 1.0587058, ...]","[-0.5531318, 0.16027026, -0.48683742, 0.2951706, -0.5182193, 0.052434415, -0.10326098, -1.1064764, 0.37777865, 0.39373252, 0.38947615, -0.5074836, 0.62014693, -0.25353014, 0.29806983, -0.38374975, 0.36246124, -0.14680131, -0.14534412, 0.031055683, -0.11341088, 0.06288045, 0.141539, -0.339424, -0.009101639, 0.2568645, -0.2690439, -0.2006097, -0.21581353, 0.14957626, 0.15211987, 0.22838588, -0.14624989, -0.124020696, -0.6190992, -0.2951612, -0.011271886, 0.24980156, -0.23616475, -0.23461491, -0.26393172, -0.2659961, -0.2944844, 0.49428883, 0.43954688, -0.19474587, 0.027549759, -0.18032664, -0.26314127, 0.045581143, 0.07658929, -0.4105302, -0.11526471, -0.2673871, -0.12131719, 0.19915521, -0.2604302, -0.10363582, -0.107538916, 0.1527329, 0.076188356, -0.38125554, 0.29528502, 0.09472762, -0.15539111, -0.2507307, -0.07677517, -0.063536294, -0.18434253, -0.19891155, 0.04470257, 0.022209646, -0.1614833, 0.22549817, 0.11542391, -0.07204664, 0.27711573, 0.047256205, 0.17023315, -0.06887744, 0.26570502, -0.014571703, 0.1640958, 0.09986951, -0.07733063, 0.057609234, -0.4284487, 0.033188056, 0.7931183, 0.12731093, -0.010633021, -0.027210422, -1.0682269, 0.39175278, -0.1340103, 0.11624803, 0.066766635, 0.37909842, 0.16639799, -0.73498434, ...]",/Users/asiegel/.pixeltable/file_cache/26f958802a424414b408646538f72715_0_3c5b8031c7610e17a42ab6df79e614c2ee1a85dbd022497373d574cf65e15c51
1,Search Search,,{},0,"[-0.4388815, 0.2616052, -0.09626006, 0.27324373, 0.32822007, 0.16367495, 0.45697403, -0.35676295, 0.134427, -0.1315698, 0.8494229, -0.107490435, -0.015755985, 0.112038106, -0.07407861, -0.6957965, 0.14700912, 0.06417219, 0.40219134, -0.23525691, 0.51686716, 0.096137054, -0.31232864, -0.5254353, 0.048178654, -0.31934243, -0.6444056, -0.48666033, 0.52254075, -0.3103736, 0.3446966, -0.21178645, 0.23859325, 0.10162328, 0.8573472, -0.18241832, -0.34733444, -0.20165437, 0.15406382, -0.19690192, 0.10764988, -0.30204758, -0.06345734, 0.8638339, -0.037420724, -0.022490362, -0.6369642, 0.4802386, 0.29758453, 0.42319107, -0.8741574, -0.4897521, -0.45491284, -0.12441757, 0.07507527, 0.23504484, -0.07021256, -0.17961068, 0.2916925, -0.48828724, 0.9852896, 0.26137888, -0.7030126, 0.31352597, -0.005079925, 0.052201174, -0.1211284, 0.006912276, 0.40216282, 0.22954431, 0.5131579, 0.26010892, -0.85585743, 0.7217986, -0.8300178, 0.31207216, -0.07977496, -0.49203598, -0.018743463, -0.31410235, -0.2637596, -0.31347972, 0.0461054, 0.13912694, -0.08210782, -0.28269756, 0.22598626, 0.32890493, 0.6892837, -0.17732784, -0.13792439, -0.44667035, -0.5042677, -0.42169738, -0.88269377, -0.058416136, 0.114198565, 0.17162606, 0.64845335, 1.0464563, ...